## Defining directory and file paths

In [2]:
data_dir = '../data'
raw_data_file = '../data/home_and_kitchen.csv'
processed_data_file = '../data/home_and_kitchen_processed.csv'

## Importing necessary libraries for data processing 

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
# read the data with coloumns by giving columns to the data frame
# data frame = home_kitchen_data 
home_kitchen_data = None
home_kitchen_data = pd.read_csv(processed_data_file)
home_kitchen_data

,productId,reviewerId,rating
0,0006564224,A3NSN9WOX8470M,5.0
1,0006564224,A2AMX0AJ2BUDNV,5.0
2,0560467893,A8LUWTIPU9CZB,5.0
3,0560467893,AABKIIHAL0L66,4.0
4,0560467893,A3DA0KIQ5OBK5C,3.0
...,...,...,...
21928563,B01HJH7K3Q,A33Z5MOHLFIECI,3.0
21928564,B01HJH7K3Q,A1BP93Y9L2HUL7,5.0
21928565,B01HJH7K3Q,A3EYW1FBJ48SH1,5.0
21928566,B01HJHTC6O,AYOGJY5CDIY49,5.0


In [5]:
from surprise.model_selection import train_test_split
from surprise import NormalPredictor
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate

## Defining the rating scale 

In [6]:
reader = Reader(rating_scale=(1, 5))

## Load Data Set 

In [7]:
data = Dataset.load_from_df(home_kitchen_data[['reviewerId', 'productId', 'rating']], reader)
data

## Use  NormalPredictor algorithm as the benchmark algorithm

In [8]:
algorithm = NormalPredictor()

## Cross Validate 

In [ ]:

cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=3, verbose=True)